In [1]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List

torch.manual_seed(0)
torch.set_float32_matmul_precision('high')

DEVICE = torch.device('cuda:0')

# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Place-holders
token_transform = {}
vocab_transform = {}


# Create source and target language tokenizer. Make sure to install the dependencies.
# pip install -U torchdata
# pip install -U spacy
# python -m spacy download en_core_web_sm
# python -m spacy download de_core_news_sm
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set UNK_IDX as the default index. This index is returned when the token is not found.
# If not set, it throws RuntimeError when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

def make_tok_masks(src, tgt):
    src_mask = (src != PAD_IDX)
    tgt_mask = (tgt != PAD_IDX)
    return src_mask, tgt_mask

In [3]:
ds = list(Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE)))

In [5]:
len(ds)

29001

In [2]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[1]
    tgt_seq_len = tgt.shape[1]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), dtype=torch.bool)

    src_padding_mask = (src == PAD_IDX)
    tgt_padding_mask = (tgt == PAD_IDX)
    return src_mask.to(DEVICE), tgt_mask.to(DEVICE), src_padding_mask.to(DEVICE), tgt_padding_mask.to(DEVICE)

# Model

In [3]:
import torch.nn as nn
from torch import Tensor

class InputEmbeddings(nn.Module):
	''' Apply learnable token and position embeddings to input tokens. '''

	def __init__(self, vocab_size: int, emb_size: int, maxlen: int = 5000):
		super().__init__()
		self.token_embedding_table = nn.Embedding(vocab_size, emb_size)
		self.position_embedding_table = nn.Embedding(maxlen, emb_size)
		self.register_buffer('pos_emb_index', torch.arange(maxlen))
	
	def forward(self, x: Tensor):
		B, T = x.shape
		tok_embd = self.token_embedding_table(x)
		pos_embd = self.position_embedding_table(self.pos_emb_index[:T])
		return tok_embd + pos_embd

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = nn.Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout,
                                       batch_first=True)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = InputEmbeddings(src_vocab_size, emb_size)
        self.tgt_tok_emb = InputEmbeddings(tgt_vocab_size, emb_size)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = (self.src_tok_emb(src))
        tgt_emb = (self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder((
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder((
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [4]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=True)
    return src_batch, tgt_batch

In [5]:
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])

EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(num_encoder_layers=NUM_ENCODER_LAYERS,
                                 num_decoder_layers=NUM_DECODER_LAYERS,
                                 emb_size=EMB_SIZE,
                                 nhead=NHEAD,
                                 src_vocab_size=SRC_VOCAB_SIZE,
                                 tgt_vocab_size=TGT_VOCAB_SIZE,
                                 dim_feedforward=FFN_HID_DIM,
                                 dropout=0.1)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [6]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:, :-1]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[:, 1:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:, :-1]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[:, 1:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [7]:
from timeit import default_timer as timer
NUM_EPOCHS = 20

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


/home/chen/.local/lib/python3.10/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched src_key_padding_mask and src_mask is deprecated. Use same type for both instead.
  warnings.warn(
/home/chen/.local/lib/python3.10/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/home/chen/.local/lib/python3.10/site-packages/torch/utils/data/datapipes/iter/combining.py:297: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


Epoch: 1, Train loss: 4.989, Val loss: 3.925, Epoch time = 7.483s
Epoch: 2, Train loss: 3.670, Val loss: 3.332, Epoch time = 7.121s
Epoch: 3, Train loss: 3.209, Val loss: 3.021, Epoch time = 6.936s
Epoch: 4, Train loss: 2.908, Val loss: 2.823, Epoch time = 6.943s
Epoch: 5, Train loss: 2.684, Val loss: 2.687, Epoch time = 6.916s
Epoch: 6, Train loss: 2.501, Val loss: 2.579, Epoch time = 6.908s
Epoch: 7, Train loss: 2.348, Val loss: 2.489, Epoch time = 6.966s
Epoch: 8, Train loss: 2.213, Val loss: 2.420, Epoch time = 7.054s
Epoch: 9, Train loss: 2.093, Val loss: 2.360, Epoch time = 7.066s
Epoch: 10, Train loss: 1.983, Val loss: 2.324, Epoch time = 7.095s
Epoch: 11, Train loss: 1.884, Val loss: 2.285, Epoch time = 7.120s
Epoch: 12, Train loss: 1.785, Val loss: 2.252, Epoch time = 7.149s
Epoch: 13, Train loss: 1.700, Val loss: 2.228, Epoch time = 7.048s
Epoch: 14, Train loss: 1.615, Val loss: 2.210, Epoch time = 7.041s
Epoch: 15, Train loss: 1.534, Val loss: 2.199, Epoch time = 7.347s
Epoc

# Lightning Training

## 1. Using Automatic Optimization

In [39]:
import pytorch_lightning as pl

class Seq2SeqTransformerLNAuto(pl.LightningModule):
      
    def __init__(self, transformer: Seq2SeqTransformer):
        super().__init__()
        self.transformer = transformer
        self.criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
    
    def forward(self, src, tgt, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask):
        return self.transformer(src, tgt, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
    
    def training_step(self, batch, batch_idx: int):
        src, tgt = batch
        tgt_input = tgt[:, :-1]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        memory_key_padding_mask = src_padding_mask
        y_pred = self(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        y_gt = tgt[:, 1:]
        loss = self.criterion(y_pred.view(-1, y_pred.shape[-1]), y_gt.reshape(-1))
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx: int):
        src, tgt = batch
        tgt_input = tgt[:, :-1]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        memory_key_padding_mask = src_padding_mask
        y_pred = self(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        y_gt = tgt[:, 1:]
        loss = self.criterion(y_pred.view(-1, y_pred.shape[-1]), y_gt.reshape(-1))
        self.log('val_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
         return torch.optim.AdamW(transformer.parameters(), lr=0.001)

In [40]:
from torch.utils.data import DataLoader

train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
train_dl = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
val_dl = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

trainer = pl.Trainer(accelerator='gpu', devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])

EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(num_encoder_layers=NUM_ENCODER_LAYERS,
                                 num_decoder_layers=NUM_DECODER_LAYERS,
                                 emb_size=EMB_SIZE,
                                 nhead=NHEAD,
                                 src_vocab_size=SRC_VOCAB_SIZE,
                                 tgt_vocab_size=TGT_VOCAB_SIZE,
                                 dim_feedforward=FFN_HID_DIM,
                                 dropout=0.1)

transformer = Seq2SeqTransformerLNAuto(transformer)

In [ ]:
trainer.fit(transformer, train_dl, val_dl)

## My Own Model

In [24]:
from dataclasses import dataclass

@dataclass
class TransformerConfig:
    block_size: int
    src_vocab_size: int 
    tgt_vocab_size: int
    n_blocks: int 
    n_heads: int 
    emb_dim: int 
    dropout: float
    bias: bool 
    weight_tying: bool
    use_grad_ckpt: bool

@dataclass
class TrainingConfig:
    learning_rate: float 
    autotune_learning_rate: bool 
    compile_model: bool
    shuffle: bool
    max_steps: int 
    batch_size: int
    gradient_accum_steps: int 
    sp_model: str
    train_src_file: str
    train_tgt_file: str
    val_src_file: str
    val_tgt_file: str

In [63]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.checkpoint import checkpoint
from torch import Tensor
from typing import *
import pytorch_lightning as pl
from dataset import TranslationBatch, TranslationDataset
import sentencepiece as sp
import math

GPU = torch.device('cuda')

@torch.jit.script
def new_gelu(x: Tensor):
	"""
	Implementation of the GELU activation function currently in Google BERT repo (identical to OpenAI GPT).
	Reference: Gaussian Error Linear Units (GELU) paper: https://arxiv.org/abs/1606.08415
	"""
	return 0.5 * x * (1.0 + torch.tanh(torch.sqrt(2.0 / torch.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

class InputEmbeddings(nn.Module):
	''' Apply learnable token and position embeddings to input tokens. '''

	def __init__(self, config: TransformerConfig, is_src = True):
		super().__init__()
		vocab_size = config.src_vocab_size if is_src else config.tgt_vocab_size
		self.token_embedding_table = nn.Embedding(vocab_size, config.emb_dim)
		self.position_embedding_table = nn.Embedding(config.block_size, config.emb_dim)
		self.register_buffer('pos_emb_index', torch.arange(config.block_size))
	
	def forward(self, x: Tensor):
		B, T = x.shape
		tok_embd = self.token_embedding_table(x)
		pos_embd = self.position_embedding_table(self.pos_emb_index[:T])
		return tok_embd + pos_embd

class MultiHeadSelfAttention(nn.Module):

	''' Multi-head self attention.
	Implements a somewhat optimized version of the self attention by combining the q, k, v projections.
	
	Inputs:
		`x`: Tensor<Float>[B, T, C] input tensor.
		`tok_mask`: Tensor<Bool>[B, T] per-token mask applied to the `x`, false is masked out, true is preserved - masks both keys and queries.

	Outputs:
		Tensor<Float>[B, T, C] output tensor.
	'''

	def __init__(self, config: TransformerConfig, is_causal: bool = False):
		super().__init__()
		self.is_causal = is_causal
		self.n_heads = config.n_heads
		self.emb_dim = config.emb_dim
		self.attn_dropout = nn.Dropout(config.dropout)
		self.resid_dropout = nn.Dropout(config.dropout)
		# combine q, k, v projections for efficiency
		self.qkv_projection = nn.Linear(config.emb_dim, 3 * config.emb_dim, bias=config.bias)
		# output projection
		self.c_proj = nn.Linear(config.emb_dim, config.emb_dim, bias=config.bias)
		if self.is_causal:
			self.register_buffer('causal_mask', torch.tril(torch.ones(config.block_size, config.block_size, dtype=torch.bool)))

	def forward(self, x: Tensor, tok_mask: Tensor):
		B, T, C = x.shape
		# proj q, k, v for all heads
		# the heads are treated as a batch dimension
		q, k, v = self.qkv_projection(x).split(self.emb_dim, dim=2)
		q = q.view(B, T, self.n_heads, C // self.n_heads).transpose(1, 2)
		k = k.view(B, T, self.n_heads, C // self.n_heads).transpose(1, 2)
		v = v.view(B, T, self.n_heads, C // self.n_heads).transpose(1, 2)
		# compute attention
		att_weights = (q @ k.transpose(-2, -1)) / math.sqrt(k.size(-1))
		mask = tok_mask.view(B, 1, 1, T) # (B, 1, 1, T) <=> (B, nh, T, T)
		if self.is_causal:
			mask = mask & self.causal_mask[None, None, :T, :T]
		att_weights = att_weights.masked_fill(mask == 0, -1e9)
		att_weights = F.softmax(att_weights, dim=-1)
		att_weights = self.attn_dropout(att_weights)
		y = att_weights @ v
		# combine heads
		y = y.transpose(1, 2).contiguous().view(B, T, C)
		y = self.resid_dropout(self.c_proj(y))
		return y

class MultiHeadCrossAttention(nn.Module):

	''' Multi-head cross attention.
	Implements a somewhat optimized version of the cross attention by combining the k, v projections.
	
	Inputs:
		`x_q`: Tensor<Float>[B, T_q, C] query input tensor.
		`x_kv`: Tensor<Float>[B, T_kv, C] key and value input tensor.
		`q_tok_mask`: Tensor<Bool>[B, T_q] mask applied to the `x_q`, false is masked out, true is preserved - applies to q only.
		`kv_tok_mask`: Tensor<Bool>[B, T_kv] mask applied to the `x_kv`, false is masked out, true is preserved - applies to k and v.

	Outputs:
		Tensor<Float>[B, T_q, C] output tensor.
	'''

	def __init__(self, config: TransformerConfig):
		super().__init__()
		self.n_heads = config.n_heads
		self.emb_dim = config.emb_dim
		self.attn_dropout = nn.Dropout(config.dropout)
		self.resid_dropout = nn.Dropout(config.dropout)
		self.q_projection = nn.Linear(config.emb_dim, config.emb_dim, bias=config.bias)
		# combine k, v projections for efficiency
		self.kv_projection = nn.Linear(config.emb_dim, 2 * config.emb_dim, bias=config.bias)
		# output projection
		self.c_proj = nn.Linear(config.emb_dim, config.emb_dim, bias=config.bias)

	def forward(self, x_q: Tensor, x_kv: Tensor, q_tok_mask: Tensor, kv_tok_mask: Tensor):
		# proj query for all heads
		B, T_q, C = x_q.shape
		q = self.q_projection(x_q)
		q = q.view(B, T_q, self.n_heads, C // self.n_heads).transpose(1, 2)
		# proj key & value for all heads
		B, T_kv, C = x_kv.shape
		k, v = self.kv_projection(x_kv).split(self.emb_dim, dim=2)
		k = k.view(B, T_kv, self.n_heads, C // self.n_heads).transpose(1, 2)
		v = v.view(B, T_kv, self.n_heads, C // self.n_heads).transpose(1, 2)
		# compute attention
		att_weights = (q @ k.transpose(-2, -1)) / math.sqrt(k.size(-1))
		# merge masks
		q_tok_mask = q_tok_mask.unsqueeze(2) # (N, T_q, 1)
		kv_tok_mask = kv_tok_mask.unsqueeze(1) # (N, 1, T_kv)
		attn_mask = q_tok_mask & kv_tok_mask
		# apply mask
		att_weights = att_weights.masked_fill(attn_mask.unsqueeze(1) == 0, -1e9)
		att_weights = F.softmax(att_weights, dim=-1)
		att_weights = self.attn_dropout(att_weights)
		y = att_weights @ v
		# combine heads
		y = y.transpose(1, 2).contiguous().view(B, T_q, C)
		y = self.resid_dropout(self.c_proj(y))
		return y

class FeedFoward(nn.Module):

	def __init__(self, config: TransformerConfig):
		super().__init__()
		self.net = nn.Sequential(
			nn.Linear(config.emb_dim, 4 * config.emb_dim),
			nn.ReLU(),
			nn.Linear(4 * config.emb_dim, config.emb_dim),
		)
		if config.dropout:
			self.net.append(nn.Dropout(config.dropout))

	def forward(self, x: Tensor):
		return self.net(x)

class EncoderBlock(nn.Module):

	def __init__(self, config: TransformerConfig):
		super().__init__()
		self.sa_module = MultiHeadSelfAttention(config)
		self.fw_module = FeedFoward(config)
		self.ln1 = nn.LayerNorm(config.emb_dim)
		self.ln2 = nn.LayerNorm(config.emb_dim)
	
	def forward(self, src: Tensor, src_mask: Tensor):
		x = src + self.sa_module(self.ln1(src), src_mask)
		x = x + self.fw_module(self.ln2(src))
		return x

class Encoder(nn.Module):

	def __init__(self, config: TransformerConfig):
		super().__init__()
		self.input_embeddings = InputEmbeddings(config)
		self.blocks = nn.ModuleList([EncoderBlock(config) for _ in range(config.n_blocks)])
		self.use_grad_ckpt = config.use_grad_ckpt
	
	def forward(self, src: Tensor, src_mask: Tensor):
		x = self.input_embeddings(src)
		for block in self.blocks:
			if self.use_grad_ckpt:
				forward = lambda *inputs: block(*inputs)
				x = checkpoint(forward, x, src_mask, preserve_rng_state=False)
			else:
				x = block(x, src_mask)
		return x

class DecoderBlock(nn.Module):

	def __init__(self, config: TransformerConfig):
		super().__init__()
		self.sa_module = MultiHeadSelfAttention(config, is_causal=True)
		self.ca_module = MultiHeadCrossAttention(config)
		self.fw_module = FeedFoward(config)
		self.ln1 = nn.LayerNorm(config.emb_dim)
		self.ln2 = nn.LayerNorm(config.emb_dim)
		self.ln3 = nn.LayerNorm(config.emb_dim)
	
	def forward(self, src: Tensor, tgt: Tensor, src_mask: Tensor, tgt_mask: Tensor):
		x = tgt + self.sa_module(self.ln1(tgt), tgt_mask)
		x = x + self.ca_module(self.ln2(tgt), self.ln2(src), tgt_mask, src_mask)
		x = x + self.fw_module(self.ln3(x))
		return x

class Decoder(nn.Module):

	def __init__(self, config: TransformerConfig):
		super().__init__()
		self.input_embeddings = InputEmbeddings(config)
		self.blocks = nn.ModuleList([DecoderBlock(config) for _ in range(config.n_blocks)])
		self.use_grad_ckpt = config.use_grad_ckpt
	
	def forward(self, src: Tensor, tgt: Tensor, src_mask: Tensor, tgt_mask: Tensor):
		x = self.input_embeddings(tgt)
		for block in self.blocks:
			if self.use_grad_ckpt:
				forward = lambda *inputs: block(*inputs)
				x = checkpoint(forward, src, x, src_mask, tgt_mask, preserve_rng_state=False)
			else:
				x = block(src, x, src_mask, tgt_mask)
		return x

class LMHead(nn.Module):

	def __init__(self, config: TransformerConfig):
		super().__init__()
		self.ln = nn.LayerNorm(config.emb_dim)
		self.logits_head = nn.Linear(config.emb_dim, config.tgt_vocab_size)
	
	def forward(self, x: Tensor):
		x = self.ln(x)
		x = self.logits_head(x)
		return x

import pytorch_lightning as pl

class TransformerModelLN(pl.LightningModule):

	def __init__(self, model_config: TransformerConfig, training_config: TrainingConfig):
		super().__init__()
		self.save_hyperparameters()
		# self.tokenizer = sp.SentencePieceProcessor(model_file=training_config.sp_model)
		self.model_config = model_config
		self.learning_rate = training_config.learning_rate
		self.batch_size = training_config.batch_size
		self.criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
		# setup sample input for tracing
		self.example_input_array = (torch.zeros(1, model_config.block_size, dtype=torch.long),
			      					torch.zeros(1, model_config.block_size, dtype=torch.long),
									torch.zeros(1, 1, model_config.block_size, dtype=torch.long),
									torch.zeros(1, 1, model_config.block_size, dtype=torch.long))
		# model parts
		self.encoder = Encoder(model_config)
		self.decoder = Decoder(model_config)
		self.lm_head = LMHead(model_config)
		# weight tying
		if model_config.weight_tying:
			self.decoder.input_embeddings.token_embedding_table.weight = self.lm_head.logits_head.weight
		# self.optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

	def forward(self, src: Tensor, tgt: Tensor, src_tok_mask: Tensor, tgt_tok_mask: Tensor):
		enc = self.encoder(src, src_tok_mask)
		dec = self.decoder(enc, tgt, src_tok_mask, tgt_tok_mask)
		logits = self.lm_head(dec)
		return logits
	
	def calculate_loss(self, y_pred: Tensor, y_true: Tensor, prefix: str):
		''' Calculate and log loss for a batch of predictions.'''
		B, T = y_true.shape
		loss = self.criterion(y_pred.view(B * T, -1), y_true.reshape(B * T))
		# log the loss
		self.log(f'{prefix}_loss', loss)
		return loss

	def calculate_metrics(self, y_pred: Tensor, y_true: Tensor, prefix: str):
		''' Calculate and log [accuracy] for a batch of predictions.'''
		B, T = y_true.shape
		# flatten the tensors
		y_pred = y_pred.view(B * T, -1).argmax(dim=-1)
		y_true = y_true.reshape(B * T)
		# calculate the metrics
		accuracy = (y_pred == y_true).float().mean()
		# log the metrics
		self.log(f'{prefix}_accuracy', accuracy)

	def training_step(self, batch: TranslationBatch, batch_idx: int):
		# opt = self.optimizer
		# opt.zero_grad()
		y_pred = self(batch.x_src, batch.x_tgt, batch.x_src_mask, batch.x_tgt_mask)
		self.calculate_metrics(y_pred, batch.y_tgt, 'train')
		loss = self.calculate_loss(y_pred, batch.y_tgt, 'train')
		# loss.backward()
		# opt.step()
		self.log('train_loss', loss)
		return loss

	def validation_step(self, batch: TranslationBatch, batch_idx: int):
		y_pred = self(batch.x_src, batch.x_tgt, batch.x_src_mask, batch.x_tgt_mask)
		self.calculate_metrics(y_pred, batch.y_tgt, 'val')
		loss = self.calculate_loss(y_pred, batch.y_tgt, 'val')
		self.log('val_loss', loss)
		return loss

	def test_step(self, batch: TranslationBatch, batch_idx: int):
		y_pred = self(batch.x_src, batch.x_tgt, batch.x_src_mask, batch.x_tgt_mask)
		self.calculate_metrics(y_pred, batch.y_tgt, 'test')
		return self.calculate_loss(y_pred, batch.y_tgt, 'test')

	def configure_optimizers(self):
		return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

In [64]:
# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True).cuda()
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=True).cuda()
    return TranslationBatch(src_batch, tgt_batch[:, :-1], src_batch != PAD_IDX, tgt_batch[:, :-1] != PAD_IDX, tgt_batch[:, 1:])

In [65]:
model_config = TransformerConfig(
    block_size= 256,
    src_vocab_size= SRC_VOCAB_SIZE,
    tgt_vocab_size= TGT_VOCAB_SIZE,
    n_blocks= 3,
    n_heads= 8, # Number of heads in multi-head attention, each head has dim = emb_dim/n_heads
    emb_dim= 512,
    dropout= 0.1, # Applies to: post-attention, post-linear
    bias= False, # Applies to: qkv projections in attention layers
    weight_tying= False,
    use_grad_ckpt= False,
)

training_config = TrainingConfig(
    learning_rate= 0.001,
    autotune_learning_rate= True,
    compile_model= False,
    shuffle= True,
    max_steps= 500000,
    batch_size= 128,
    gradient_accum_steps= 1,
    sp_model= 'data/multi30k/m_en_de.model',
    # training
    train_src_file= 'data/multi30k/train.en',
    train_tgt_file= 'data/multi30k/train.de',
    # validation
    val_src_file= 'data/multi30k/val.en',
    val_tgt_file= 'data/multi30k/val.de',
)

In [66]:
model = TransformerModelLN(model_config, training_config)

In [67]:
from torch.utils.data import DataLoader

train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
train_dl = DataLoader(train_iter, batch_size=training_config.batch_size, shuffle=training_config.shuffle, collate_fn=collate_fn)
val_dl = DataLoader(val_iter, batch_size=training_config.batch_size, shuffle=training_config.shuffle, collate_fn=collate_fn)

In [68]:
trainer = pl.Trainer(gpus=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [69]:
trainer.fit(model, train_dl, val_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | In sizes                                            | Out sizes      
-----------------------------------------------------------------------------------------------------------------------
0 | criterion | CrossEntropyLoss | 0      | ?                                                   | ?              
1 | encoder   | Encoder          | 19.4 M | [[1, 256], [1, 1, 256]]                             | [1, 256, 512]  
2 | decoder   | Decoder          | 22.6 M | [[1, 256, 512], [1, 256], [1, 1, 256], [1, 1, 256]] | [1, 256, 512]  
3 | lm_head   | LMHead           | 5.6 M  | [1, 256, 512]                                       | [1, 256, 10837]
-----------------------------------------------------------------------------------------------------------------------
47.5 M    Trainable params
0         Non-trainable params
47.5 M    Total params
190.194   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [71]:
trainer.validate(model, val_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.33583131432533264
        val_loss             2.153543472290039
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_accuracy': 0.33583131432533264, 'val_loss': 2.153543472290039}]